In [1]:
!gdown 1viFKqtYTtTiP9_EdBXVpCmWbNmxDiXWG

Downloading...
From (original): https://drive.google.com/uc?id=1viFKqtYTtTiP9_EdBXVpCmWbNmxDiXWG
From (redirected): https://drive.google.com/uc?id=1viFKqtYTtTiP9_EdBXVpCmWbNmxDiXWG&confirm=t&uuid=a2fbed11-21c3-4712-bdf5-8de7ded99cf5
To: /content/intern_task.csv
100% 222M/222M [00:01<00:00, 121MB/s]


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('intern_task.csv')

In [4]:
df.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,273.0,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000


In [5]:
df['query_id'].nunique()

2000

In [8]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.7 MB/s eta 0:00:00


In [9]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import numpy as np
import os

In [16]:
from sklearn.model_selection import GroupShuffleSplit
df['query_id'] = df['query_id'].astype('category')
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Разделяем датафрейм на обучающий и тестовый наборы

train_idx, test_idx = next(gss.split(df, groups=df['query_id']))

# Создаем обучающий и тестовый датафреймы
train_df = df.iloc[train_idx]
test_df = df.iloc[test_idx]


In [101]:
train_df['query_id'] = train_df['query_id'].astype(int)
test_df['query_id'] = test_df['query_id'].astype(int)

<ipython-input-101-315393e75971>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['query_id'] = train_df['query_id'].astype(int)
<ipython-input-101-315393e75971>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['query_id'] = test_df['query_id'].astype(int)


In [103]:
X_train = train_df.drop(['rank', 'query_id'], axis=1).values
y_train = train_df['rank'].values
queries_train = train_df['query_id'].values

X_test = test_df.drop(['rank', 'query_id'], axis=1).values
y_test = test_df['rank'].values
queries_test = test_df['query_id'].values

In [104]:
from collections import Counter
Counter(y_train).items()

dict_items([(0, 97647), (1, 60601), (2, 25863), (4, 1473), (3, 3405)])

In [106]:
max_relevance = np.max(y_train)
max_relevance = np.float64(max_relevance)
y_train = y_train.astype(np.float64)
y_test = y_test.astype(np.float64)
y_train /= max_relevance
y_test /= max_relevance

In [107]:
num_queries = np.unique(queries_train).shape[0]
num_queries

1600

In [39]:
from catboost import CatBoostRanker, Pool, MetricVisualizer

In [108]:
train = Pool(
    data=X_train,
    label=y_train,
    group_id=queries_train
)

test = Pool(
    data=X_test,
    label=y_test,
    group_id=queries_test
)

In [129]:
default_parameters = {
    'iterations': 2000,
    'verbose': 100,
    'random_seed': 0,
}

parameters = {}

In [130]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function

    if additional_params is not None:
        parameters.update(additional_params)

    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)

    return model

In [219]:
model = fit_model('RMSE', {'custom_metric':['PrecisionAt:top=5', 'RecallAt:top=5', 'MAP:top=5','NDCG:top=5'],
                               'task_type':'GPU'
                               })

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.089779
0:	learn: 0.2026595	test: 0.1997178	best: 0.1997178 (0)	total: 30.6ms	remaining: 1m 1s


Default metric period is 5 because PrecisionAt, RecallAt, MAP, NDCG is/are not implemented for GPU
Metric PrecisionAt:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric RecallAt:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


100:	learn: 0.1687124	test: 0.1686420	best: 0.1686420 (100)	total: 1.45s	remaining: 27.3s
200:	learn: 0.1669414	test: 0.1678065	best: 0.1678065 (200)	total: 2.49s	remaining: 22.3s
300:	learn: 0.1656805	test: 0.1673491	best: 0.1673491 (300)	total: 3.89s	remaining: 21.9s
400:	learn: 0.1646128	test: 0.1670520	best: 0.1670520 (400)	total: 7.17s	remaining: 28.6s
500:	learn: 0.1636552	test: 0.1668438	best: 0.1668438 (500)	total: 13s	remaining: 38.8s
600:	learn: 0.1627991	test: 0.1666537	best: 0.1666537 (600)	total: 15.8s	remaining: 36.8s
700:	learn: 0.1619914	test: 0.1664778	best: 0.1664778 (700)	total: 16.8s	remaining: 31.1s
800:	learn: 0.1612429	test: 0.1663457	best: 0.1663457 (800)	total: 17.8s	remaining: 26.6s
900:	learn: 0.1605360	test: 0.1662526	best: 0.1662522 (890)	total: 18.8s	remaining: 22.9s
1000:	learn: 0.1598622	test: 0.1661365	best: 0.1661365 (1000)	total: 20.6s	remaining: 20.5s
1100:	learn: 0.1592198	test: 0.1660759	best: 0.1660744 (1090)	total: 24.3s	remaining: 19.8s
1200:	le

In [220]:
eval_metrics = model.eval_metrics(test, ['NDCG:top=5','MAP:top=5'], plot=True)
print('Результаты для модели на RMSE')
print('NDCG@5 для последних 5 итераций: ', eval_metrics['NDCG:top=5;type=Base'][:5])
print('mAP@5 для последних 5 итераций: ',eval_metrics['MAP:top=5'][:5])

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Результаты для модели на RMSE
NDCG@5 для последних 5 итераций:  [0.48159185327584547, 0.5467482186998036, 0.5732792545798461, 0.6109417702944671, 0.6238541530080286]
mAP@5 для последних 5 итераций:  [0.1392354166666667, 0.17169722222222233, 0.18683888888888897, 0.2072152777777778, 0.21121805555555562]


In [215]:
model1 = fit_model('YetiRank', {'custom_metric':['PrecisionAt:top=5', 'RecallAt:top=5', 'MAP:top=5','NDCG:top=5'],
                               'task_type':'GPU'
                               })

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because PFound, PrecisionAt, RecallAt, MAP, NDCG is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PrecisionAt:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric RecallAt:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.6022369	best: 0.6022369 (0)	total: 43.7ms	remaining: 1m 27s
100:	test: 0.7628423	best: 0.7635716 (80)	total: 1.6s	remaining: 30.2s
200:	test: 0.7718750	best: 0.7718750 (200)	total: 3.07s	remaining: 27.5s
300:	test: 0.7735179	best: 0.7735632 (285)	total: 4.48s	remaining: 25.3s
400:	test: 0.7735067	best: 0.7738983 (380)	total: 7.09s	remaining: 28.3s
500:	test: 0.7734673	best: 0.7740544 (435)	total: 9.72s	remaining: 29.1s
600:	test: 0.7737829	best: 0.7742235 (575)	total: 11.2s	remaining: 26s
700:	test: 0.7747066	best: 0.7751785 (660)	total: 12.6s	remaining: 23.4s
800:	test: 0.7747646	best: 0.7751785 (660)	total: 14.1s	remaining: 21s
900:	test: 0.7748959	best: 0.7751785 (660)	total: 15.5s	remaining: 19s
1000:	test: 0.7752219	best: 0.7752332 (975)	total: 17s	remaining: 17s
1100:	test: 0.7757679	best: 0.7758104 (1070)	total: 18.5s	remaining: 15.1s
1200:	test: 0.7757130	best: 0.7759512 (1170)	total: 22.3s	remaining: 14.8s
1300:	test: 0.7756266	best: 0.7759512 (1170)	total: 23.9s	re

In [216]:
eval_metrics1 = model1.eval_metrics(test, ['NDCG:top=5','MAP:top=5'], plot=True)
print('Результаты для модели на YetiRank')
print('NDCG@5 для последних 5 итераций: ', eval_metrics1['NDCG:top=5;type=Base'][:5])
print('mAP@5 для последних 5 итераций: ',eval_metrics1['MAP:top=5'][:5])

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Результаты для модели на YetiRank
NDCG@5 for 5 last iterations:  [0.4113915965751624, 0.4594006637034756, 0.5030913092219484, 0.513561430457291, 0.5102173652067452]
mAP@5 for 5 last iterations:  [0.07629305555555556, 0.09405694444444444, 0.11051388888888891, 0.1181729166666667, 0.1149875]


In [217]:
model2 = fit_model('YetiRankPairwise', {'custom_metric':['PrecisionAt:top=5', 'RecallAt:top=5', 'MAP:top=5','NDCG:top=5'],
                               'task_type':'GPU'
                               })

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because PFound, PrecisionAt, RecallAt, MAP, NDCG is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PrecisionAt:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric RecallAt:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.6072933	best: 0.6072933 (0)	total: 103ms	remaining: 3m 26s
100:	test: 0.7052603	best: 0.7052610 (95)	total: 4.39s	remaining: 1m 22s
200:	test: 0.7492778	best: 0.7492778 (200)	total: 9.48s	remaining: 1m 24s
300:	test: 0.7546876	best: 0.7546876 (300)	total: 13.4s	remaining: 1m 15s
400:	test: 0.7569615	best: 0.7573416 (340)	total: 17.6s	remaining: 1m 10s
500:	test: 0.7577917	best: 0.7586520 (440)	total: 22.8s	remaining: 1m 8s
600:	test: 0.7598921	best: 0.7599842 (590)	total: 26.9s	remaining: 1m 2s
700:	test: 0.7604461	best: 0.7605106 (690)	total: 31.3s	remaining: 58s
800:	test: 0.7616355	best: 0.7616355 (800)	total: 36.8s	remaining: 55s
900:	test: 0.7614447	best: 0.7621094 (870)	total: 41s	remaining: 50.1s
1000:	test: 0.7619233	best: 0.7621094 (870)	total: 46s	remaining: 46s
1100:	test: 0.7627752	best: 0.7629188 (1095)	total: 51.1s	remaining: 41.7s
1200:	test: 0.7631540	best: 0.7632015 (1195)	total: 55.6s	remaining: 37s
1300:	test: 0.7636479	best: 0.7637395 (1250)	total: 1m 1s	

In [218]:
eval_metrics2 = model2.eval_metrics(test, ['NDCG:top=5','MAP:top=5'], plot=True)
print('Результаты для модели на RMSE')
print('NDCG@5 для последних 5 итераций: ', eval_metrics2['NDCG:top=5;type=Base'][:5])
print('mAP@5 для последних 5 итераций: ',eval_metrics2['MAP:top=5'][:5])

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Результаты для модели на RMSE
NDCG@5 for 5 last iterations:  [0.413711562839219, 0.42036510148928985, 0.5062028382043576, 0.5070507387175416, 0.5164260997954644]
mAP@5 for 5 last iterations:  [0.08107916666666667, 0.08107916666666667, 0.12096250000000007, 0.12096250000000007, 0.12429583333333341]


Результаты тренировки CatBoost сохраняются в директории.
